In [16]:
import math
import json
%run ActorCritic.ipynb
%run Breed.ipynb
%run PrintFunctions.ipynb

In [17]:
class BracketFunctions:
    num = 128
    rankDict = {}
    stopRank = math.log2(num)
    nameList = [str(i) for i in range(num)]
    def writeWinners(name1, name2):
        formattedPrint(f"writing winners {[name1, name2]}")
        jsonDict = {"names": [name1, name2]}
        with open("bracketWinners.json", "w") as winnerFile:
            winnerFile.write(json.dumps(jsonDict))
        formattedPrint("write suceeded")
    def readWinners():
        formattedPrint("reading winners...")
        with open("bracketWinners.json", "r") as winnerFile:
            txt = winnerFile.read()
            jsonDict = json.loads(txt)
            formattedPrint(f"got json of {jsonDict}")
            return jsonDict["names"]
    def generateInitialPlayers(num = 128):
        progbar = tf.keras.utils.Progbar(num)
        formattedPrint("generating initial players...")
        start = timer()
        playerList = []
        for i in range(128):
            playerList.append(Player(name = BracketFunctions.nameList[i]))
            progbar.add(1)
        BracketFunctions.rankDict[0] = playerList
        end = timer()
        formattedPrint(f"finished generation in {end - start} seconds ")
    def generateBredPlayers(player1, player2):
        formattedPrint(f"breeding offspring from {player1.name} and {player2.name}")
        start = timer()
        playerList = []
        nets = BreedingFunctions.breed(player1.net, player2.net, BreedingFunctions.uniformSwitch)
        for i in range(BracketFunctions.num):
            playerList.append(Player(name = BracketFunctions.nameList[i], net = nets[i]))
        BracketFunctions.rankDict[0] = playerList
        end = timer()
        formattedPrint(f"finished generation in {end - start} seconds ")
    def runRanks(num = 0):
        formattedPrint(f"RUNNING RANK {num}", location = f"RANK {num}")
        if len(BracketFunctions.rankDict[num]) > 2:
            while len(BracketFunctions.rankDict[num]) >= 2:
                while True:
                    try:
                        formattedPrint(f"STARTING GAME BETWEEN {[BracketFunctions.rankDict[num][0].name, BracketFunctions.rankDict[num][1].name]}", location = f"RANK {num}")
                        start = timer()
                        with HiddenPrints():
                            winner = ActorCritic.trainStep(BracketFunctions.rankDict[num][0], 
                                                BracketFunctions.rankDict[num][1])["winner"]
                        end = timer()
                        formattedPrint(f"finished game in {end - start} seconds", location = f"RANK {num}")
                        break
                    except AssertionError:
                        formattedPrint("Controlled error occurred, resetting", location = f"RANK {num}")
                        continue
                formattedPrint(f"{BracketFunctions.rankDict[num][0 if winner == 1 else 1].name} won!", location = f"RANK {num}")
                winnerPlayer = BracketFunctions.rankDict[num][0 if winner == 1 else 1]
                winnerPlayer.setRandomTeam()
                try:
                    BracketFunctions.rankDict[num + 1].append(winnerPlayer)
                except KeyError:
                    BracketFunctions.rankDict[num + 1] = [winnerPlayer]
                del BracketFunctions.rankDict[num][0]
                del BracketFunctions.rankDict[num][0]
            BracketFunctions.runRanks(num = num + 1)
        else:
            formattedPrint("found winners, saving...")
            p1 = BracketFunctions.rankDict[num][0]
            p1.save()
            p2 = BracketFunctions.rankDict[num][1]
            p2.save()
            formattedPrint(f"WINNERS {[p1.name, p2.name]}", location = f"RANK {num}")
            BracketFunctions.writeWinners(p1.name, p2.name)
            BracketFunctions.rankDict = {}
            BracketFunctions.generateBredPlayers(p1, p2)
            BracketFunctions.runRanks(num = 0)
    def run():
        GameFunctions.load()
        names = []
        try:
            names = BracketFunctions.readWinners()
            p1 = Player(name = names[0])
            p1.load(names[0])
            p2 = Player(name = names[1])
            p2.load(names[1])
            BracketFunctions.generateBredPlayers(p1, p2)
        except FileNotFoundError:
            BracketFunctions.generateInitialPlayers()
        BracketFunctions.runRanks(num = 0)

In [20]:
BracketFunctions.run()

Mon Mar 14 07:08:21 2022, GENERAL:: reading winners...
Mon Mar 14 07:08:21 2022, GENERAL:: got json of {'names': ['0', '73']}
Mon Mar 14 07:08:22 2022, PLAYERS:: loaded player 0
Mon Mar 14 07:08:23 2022, PLAYERS:: loaded player 73
Mon Mar 14 07:08:23 2022, GENERAL:: breeding offspring from 0 and 73
128/128 [==============================] - 77s 601ms/step
Mon Mar 14 07:11:10 2022, GENERAL:: finished generation in 166.68972223500168 seconds 
Mon Mar 14 07:11:10 2022, RANK 0:: RUNNING RANK 0
Mon Mar 14 07:11:10 2022, RANK 0:: STARTING GAME BETWEEN ['0', '1']
Mon Mar 14 07:11:25 2022, RANK 0:: finished game in 15.383955830999184 seconds
Mon Mar 14 07:11:25 2022, RANK 0:: 1 won!
Mon Mar 14 07:11:26 2022, RANK 0:: STARTING GAME BETWEEN ['2', '3']
Mon Mar 14 07:11:51 2022, RANK 0:: finished game in 24.94177275599941 seconds
Mon Mar 14 07:11:51 2022, RANK 0:: 2 won!
Mon Mar 14 07:11:52 2022, RANK 0:: STARTING GAME BETWEEN ['4', '5']
Mon Mar 14 07:12:16 2022, RANK 0:: finished game in 24.15229

KeyboardInterrupt: 